# 02 Pandas Reading Excel (data.europe.eu)

Content:

* Loading (large) Excel file, 75MB
* Basic inspection


----

Note:

* Requires additional libraries to load Excel files, e.g. [xlrd](https://github.com/python-excel/xlrd) or [openpyxl](https://openpyxl.readthedocs.io/en/stable/).

In [4]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 2.2 MB/s eta 0:00:000:00:01 eta 0:00:01


In [5]:
!tree -sh data/E-PRTR_database_v13

[4.0K]  data/E-PRTR_database_v13
├── [ 75M]  Pollutant releases.xlsx
├── [7.6M]  Pollutant transfers.xlsx
└── [ 94M]  Waste transfers.xlsx

0 directories, 3 files


In [6]:
import pandas as pd

Reading 75MB of Excel can take a while (few minutes), even with Pandas.

In [7]:
df = pd.read_excel("data/E-PRTR_database_v13/Pollutant releases.xlsx")

## Basic inspection

Key takeaways:

* Pandas has inferred the types of various columns (`dtypes: bool(1), float64(3), int64(2), object(16)`)

Few inspection functions:

* `df.info`
* `df.describe`
* `df["columnname"].unique()`

Real-world data is often not complete, values are missing. That's why a `df.info()` is useful to have a first glance at the data quality.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537982 entries, 0 to 537981
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   FacilityID              537982 non-null  int64  
 1   NationalID              537982 non-null  object 
 2   FacilityName            537656 non-null  object 
 3   StreetName              533093 non-null  object 
 4   BuildingNumber          364475 non-null  object 
 5   PostalCode              537884 non-null  object 
 6   City                    537902 non-null  object 
 7   CountryName             537982 non-null  object 
 8   Lat                     537982 non-null  float64
 9   Long                    537982 non-null  float64
 10  CompetentAuthorityName  537982 non-null  object 
 11  MainIAActivityCode      537982 non-null  object 
 12  MainIASectorName        537982 non-null  object 
 13  MainIAActivityName      537982 non-null  object 
 14  MainIASubActivityNam

Then, to get an impression of the content, the column names might or might not be speaking.

In [9]:
df.columns

Index(['FacilityID', 'NationalID', 'FacilityName', 'StreetName',
       'BuildingNumber', 'PostalCode', 'City', 'CountryName', 'Lat', 'Long',
       'CompetentAuthorityName', 'MainIAActivityCode', 'MainIASectorName',
       'MainIAActivityName', 'MainIASubActivityName', 'MainActivityIndicator',
       'ReportingYear', 'PollutantName', 'PollutantGroupName', 'TotalQuantity',
       'ReleaseMediumName', 'MethodTypeName'],
      dtype='object')

With the various selection techniques, it is quite easy to just peek into the data and get more impressions of values and shape.

In [10]:
df.CountryName[:10]

0    Czech Republic
1    Czech Republic
2    Czech Republic
3    Czech Republic
4    Czech Republic
5    Czech Republic
6    Czech Republic
7    Czech Republic
8    Czech Republic
9    Czech Republic
Name: CountryName, dtype: object

Unique values are often of interest as well.

In [11]:
df.CountryName.unique()

array(['Czech Republic', 'United Kingdom', 'Ireland', 'Estonia',
       'Finland', 'Hungary', 'Lithuania', 'Luxembourg', 'Malta',
       'Romania', 'Slovenia', 'Cyprus', 'Slovakia', 'Denmark', 'France',
       'Poland', 'Greece', 'Iceland', 'Latvia', 'Serbia', 'Portugal',
       'Bulgaria', 'Norway', 'Italy', 'Austria', 'Switzerland',
       'Netherlands', 'Belgium', 'Croatia', 'Spain', 'Sweden', 'Germany'],
      dtype=object)

In [12]:
df.City.unique()

array(['Mladá Boleslav', 'Praha 18', 'Slezská Ostrava', ...,
       'SAINT-BENIGNE', 'PONT-AVEN', 'GRACE-UZEL'], dtype=object)

In [13]:
len(df.City.unique())

15104

Most of the time, unique values *per column* are what is relevant. In case the unique values should be queried
across multiple columns, one has to take a slight workaround: first concatenate the columns and then run unique on that.

In [14]:
pd.concat([df['City'], df['CountryName']]).unique()

array(['Mladá Boleslav', 'Praha 18', 'Slezská Ostrava', ..., 'Spain',
       'Sweden', 'Germany'], dtype=object)

In [15]:
len(pd.concat([df['City'], df['CountryName']]).unique())

15135

The value almost equals the sum of the unique values.

In [16]:
len(df.City.unique()) + len(df.CountryName.unique())

15136

Which means, that there is an overlap between city and country names. The interoperability of Pandas, numpy and Python makes it  quite simple to find the overlap with the built-in set data type.

In [17]:
set(df.City.unique()) & set(df.CountryName.unique())

{'Luxembourg'}

## Grouping

* Grouping is one of the most common operations to perform on data.
* The df.groupyby function is "call-by-need" or lazily evaluated, it returns a DataFrameGroupBy object.
* Aggregations, like size are realised as chained calls.

One useful stanza is the `df.groupy("columnname").size().sort_values()` expression, which ranks values by frequency.


In [18]:
df.groupby("CountryName")

In [19]:
df.groupby("CountryName").size().sort_values(ascending=False)

CountryName
Germany           95038
United Kingdom    79397
Italy             49962
Spain             48544
France            41749
Poland            34430
Norway            27525
Belgium           22077
Netherlands       19737
Czech Republic    18312
Portugal          15397
Sweden            12333
Romania           11237
Finland            9124
Hungary            9014
Slovakia           8593
Ireland            5549
Bulgaria           4766
Austria            4702
Greece             3844
Denmark            3386
Slovenia           2366
Switzerland        2366
Serbia             1989
Estonia            1574
Lithuania          1435
Cyprus              921
Luxembourg          740
Latvia              623
Iceland             523
Croatia             405
Malta               324
dtype: int64

## How many pollutants are listed?

* And which one is the most frequent?

In [20]:
len(df["PollutantName"].unique())

97

In [21]:
df.groupby("PollutantName").size().sort_values(ascending=False)

PollutantName
Ammonia (NH3)                                       67498
Nitrogen oxides (NOx/NO2)                           37951
Carbon dioxide (CO2)                                33335
Zinc and compounds (as Zn)                          32970
Total organic carbon (TOC) (as total C or COD/3)    25159
                                                    ...  
Chlordane                                               6
Inorganic substances                                    3
Other organic substances                                2
Mirex                                                   2
Heavy metals                                            1
Length: 97, dtype: int64

## Which pollutant groups are listed?

In [22]:
df.PollutantGroupName.unique()

array(['Other gases', 'Greenhouse gases', 'Other organic substances',
       'Heavy metals', 'Inorganic substances',
       'Chlorinated organic substances', 'Pesticides'], dtype=object)

## The medium of release for the pollutant has fewer manifestations.

* Air
* Water
* Soil

In [23]:
df.ReleaseMediumName.unique()

array(['Air', 'Water', 'Soil'], dtype=object)

In [24]:
df.groupby(df.ReleaseMediumName).size().sort_values(ascending=False)

ReleaseMediumName
Air      316050
Water    218721
Soil       3211
dtype: int64

## Grouping by more than one attribute.

Grouping by more than one attribute is supported, by passing a list of columns to `df.groupby`.

Are there pollutants, that are release through more than one medium?

In [25]:
nm = df.groupby([df.PollutantName, df.ReleaseMediumName]).size()

The result is a series with a hierarchical index.

> The MultiIndex object is the hierarchical analogue of the standard Index object which typically stores the axis labels in pandas objects. You can think of MultiIndex as an array of tuples where each tuple is unique.

From: https://pandas.pydata.org/pandas-docs/stable/advanced.html

In [26]:
nm.head(20)

PollutantName                            ReleaseMediumName
1,1,1-trichloroethane                    Air                    262
                                         Soil                     5
                                         Water                   33
1,1,2,2-tetrachloroethane                Air                    111
1,2,3,4,5,6-hexachlorocyclohexane (HCH)  Air                      1
                                         Water                  108
1,2-dichloroethane (DCE)                 Air                    739
                                         Soil                     4
                                         Water                 1019
Alachlor                                 Water                   29
Aldrin                                   Air                      1
                                         Water                  111
Ammonia (NH3)                            Air                  67478
                                         Water           

In [27]:
type(nm), type(nm.index)

(pandas.core.series.Series, pandas.core.indexes.multi.MultiIndex)

In [28]:
nm["Aldrin"]

ReleaseMediumName
Air        1
Water    111
dtype: int64

In [29]:
nm["Aldrin", "Air"]

1

To sort an index on various levels, we can use `sort_index`.

In [30]:
nm.sort_index(level=0, ascending=True).head()

PollutantName                            ReleaseMediumName
1,1,1-trichloroethane                    Air                  262
                                         Soil                   5
                                         Water                 33
1,1,2,2-tetrachloroethane                Air                  111
1,2,3,4,5,6-hexachlorocyclohexane (HCH)  Air                    1
dtype: int64

In [31]:
nm.sort_index(level=1, ascending=True)

PollutantName                            ReleaseMediumName
1,1,1-trichloroethane                    Air                    262
1,1,2,2-tetrachloroethane                Air                    111
1,2,3,4,5,6-hexachlorocyclohexane (HCH)  Air                      1
1,2-dichloroethane (DCE)                 Air                    739
Aldrin                                   Air                      1
                                                              ...  
Trifluralin                              Water                   15
Triphenyltin and compounds               Water                   56
Vinyl chloride                           Water                  289
Xylenes                                  Water                 1173
Zinc and compounds (as Zn)               Water                25392
Length: 177, dtype: int64

We sort by the second level (using level=1). The result is still a series, the display just looks more DataFrame-like.

In [32]:
by_medium = nm.sort_index(level=1, ascending=True)

In [33]:
by_medium.head()

PollutantName                            ReleaseMediumName
1,1,1-trichloroethane                    Air                  262
1,1,2,2-tetrachloroethane                Air                  111
1,2,3,4,5,6-hexachlorocyclohexane (HCH)  Air                    1
1,2-dichloroethane (DCE)                 Air                  739
Aldrin                                   Air                    1
dtype: int64

Access by chaining.

In [34]:
by_medium["Aldrin"]["Water"]

111

In [35]:
nm

PollutantName                            ReleaseMediumName
1,1,1-trichloroethane                    Air                    262
                                         Soil                     5
                                         Water                   33
1,1,2,2-tetrachloroethane                Air                    111
1,2,3,4,5,6-hexachlorocyclohexane (HCH)  Air                      1
                                                              ...  
Xylenes                                  Air                     13
                                         Water                 1173
Zinc and compounds (as Zn)               Air                   7016
                                         Soil                   562
                                         Water                25392
Length: 177, dtype: int64

The idea, that the indices behave like tuples, can be observed here as well.

In [36]:
nm[("Anthracene",)]

ReleaseMediumName
Air      257
Soil      10
Water    817
dtype: int64

In [37]:
nm[("Anthracene", "Water")]

817

## Masking on hierarchical index.

To filter all levels, we can use boolean indexing as usual.

In [38]:
nm[nm > 100]

PollutantName                            ReleaseMediumName
1,1,1-trichloroethane                    Air                    262
1,1,2,2-tetrachloroethane                Air                    111
1,2,3,4,5,6-hexachlorocyclohexane (HCH)  Water                  108
1,2-dichloroethane (DCE)                 Air                    739
                                         Water                 1019
                                                              ...  
Vinyl chloride                           Water                  289
Xylenes                                  Water                 1173
Zinc and compounds (as Zn)               Air                   7016
                                         Soil                   562
                                         Water                25392
Length: 108, dtype: int64

To restrict the value for a given type, we can then filter by e.g. PollutantName. This results in a series that contains all pollutants released through water which have measurements above 1000.

In [39]:
nm[nm > 1000][:, "Water"]

PollutantName
1,2-dichloroethane (DCE)                              1019
Arsenic and compounds (as As)                        10191
Asbestos                                              1107
Benzene                                               1325
Cadmium and compounds (as Cd)                         5159
Chlorides (as total Cl)                               9768
Chromium and compounds (as Cr)                        6317
Copper and compounds (as Cu)                         12883
Cyanides (as total CN)                                3694
Di-(2-ethyl hexyl) phthalate (DEHP)                   3146
Dichloromethane (DCM)                                 1216
Diuron                                                1114
Fluorides (as total F)                                7630
Halogenated organic compounds (as AOX)                5823
Lead and compounds (as Pb)                            8855
Mercury and compounds (as Hg)                         6379
Naphthalene                               

Another way would be to use `get_level_values`.

> Return an Index of values for requested level, equal to the length of the index.

It is important to keep the index length intact.

In [40]:
nm[(nm.index.get_level_values("ReleaseMediumName") == "Water") & (nm > 1000)]

PollutantName                                      ReleaseMediumName
1,2-dichloroethane (DCE)                           Water                 1019
Arsenic and compounds (as As)                      Water                10191
Asbestos                                           Water                 1107
Benzene                                            Water                 1325
Cadmium and compounds (as Cd)                      Water                 5159
Chlorides (as total Cl)                            Water                 9768
Chromium and compounds (as Cr)                     Water                 6317
Copper and compounds (as Cu)                       Water                12883
Cyanides (as total CN)                             Water                 3694
Di-(2-ethyl hexyl) phthalate (DEHP)                Water                 3146
Dichloromethane (DCM)                              Water                 1216
Diuron                                             Water                 

## Looking at a specific location

How many German cities are contained in the dataset?

In [41]:
df[df.CountryName == "Germany"].City.unique().size

1751

How many measurements per city in Germany?

In [42]:
df[df.CountryName == "Germany"].groupby("City").size().sort_values(ascending=False)

City
Ludwigshafen am Rhein                  7103
Duisburg                               4802
Wesseling                              2210
Burghausen i.Lkr. Altötting            1896
Köln                                   1770
                                       ... 
Oschersleben (Bode)                       1
Oschersleben (Bode) OT Hadmersleben       1
Laußig                                    1
Oschersleben OT Hadmersleben              1
Leisnig                                   1
Length: 1750, dtype: int64

Is a certain city contained?

In [54]:
"Dresden" in df[df.CountryName == "Germany"].City.values

True

How many entries for given city?

In [53]:
df[df.City == "Dresden"].size

4268

Filtering out the FacilityName and StreetName for entries in a given city.

In [56]:
df[df.City == "Dresden"][["FacilityName", "StreetName"]].head(20)

,FacilityName,StreetName
436710,Kläranlage Dresden-Kaditz,Scharfenberger Str.
436711,Kläranlage Dresden-Kaditz,Scharfenberger Str.
436712,Kläranlage Dresden-Kaditz,Scharfenberger Str.
436713,Kläranlage Dresden-Kaditz,Scharfenberger Str.
436714,Kläranlage Dresden-Kaditz,Scharfenberger Str.
436715,Kläranlage Dresden-Kaditz,Scharfenberger Str.
436716,Kläranlage Dresden-Kaditz,Scharfenberger Str.
436717,Kläranlage Dresden-Kaditz,Scharfenberger Str.
436718,Kläranlage Dresden-Kaditz,Scharfenberger Str.
436719,Kläranlage Dresden-Kaditz,Scharfenberger Str.


Facilities with most entries for a given city.

In [51]:
df[df.City == "Dresden"].groupby("FacilityName").size().sort_values(ascending=False)

FacilityName
Kläranlage Dresden-Kaditz                           164
DREWAG Gasturbinen-Heizkraftwerk Nossener Brücke     20
SRD Deponie Radeburger Straße                         8
Holzheizkraftwerk Dresden                             1
ZAOE Hausmülldeponie Langebrücker/Dresdner Str.       1
dtype: int64

In [47]:
df[df.City == "Dresden"].groupby("FacilityName").size().sort_values(ascending=False)

FacilityName
Kläranlage Dresden-Kaditz                           164
DREWAG Gasturbinen-Heizkraftwerk Nossener Brücke     20
SRD Deponie Radeburger Straße                         8
Holzheizkraftwerk Dresden                             1
ZAOE Hausmülldeponie Langebrücker/Dresdner Str.       1
dtype: int64

Sort by pollutant groups.

In [49]:
df[df.City == "Dresden"].groupby(["FacilityName", "PollutantGroupName"]).size().sort_index(level=1)

FacilityName                                      PollutantGroupName            
Kläranlage Dresden-Kaditz                         Chlorinated organic substances    17
DREWAG Gasturbinen-Heizkraftwerk Nossener Brücke  Greenhouse gases                  10
SRD Deponie Radeburger Straße                     Greenhouse gases                   8
ZAOE Hausmülldeponie Langebrücker/Dresdner Str.   Greenhouse gases                   1
Kläranlage Dresden-Kaditz                         Heavy metals                      87
                                                  Inorganic substances              43
DREWAG Gasturbinen-Heizkraftwerk Nossener Brücke  Other gases                       10
Holzheizkraftwerk Dresden                         Other organic substances           1
Kläranlage Dresden-Kaditz                         Other organic substances          12
                                                  Pesticides                         5
dtype: int64

Sort by values.

In [50]:
df[df.City == "Dresden"].groupby(["FacilityName", "PollutantGroupName"]).size().sort_values(ascending=False)

FacilityName                                      PollutantGroupName            
Kläranlage Dresden-Kaditz                         Heavy metals                      87
                                                  Inorganic substances              43
                                                  Chlorinated organic substances    17
                                                  Other organic substances          12
DREWAG Gasturbinen-Heizkraftwerk Nossener Brücke  Greenhouse gases                  10
                                                  Other gases                       10
SRD Deponie Radeburger Straße                     Greenhouse gases                   8
Kläranlage Dresden-Kaditz                         Pesticides                         5
Holzheizkraftwerk Dresden                         Other organic substances           1
ZAOE Hausmülldeponie Langebrücker/Dresdner Str.   Greenhouse gases                   1
dtype: int64